# Библиотеки

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose

from src.experiments import ExperimentTracker
from src.metrics import evaluate
from src.models import BaselineYearAgo, aggregated_daily_predictions
from src.plots import plot_prediction, plot_target_boxplots
from src.process_data import prepare_dataset, prepare_parsed_weather, read_datasets

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
plt.rcParams["figure.figsize"] = (5, 5)
plt.rcParams["figure.dpi"] = 100

# Загрузка данных

In [ ]:
datasets = read_datasets()

In [ ]:
datasets.keys()

In [ ]:
tracker = ExperimentTracker()

In [ ]:
train = datasets["source_train"].copy()
test = datasets["source_test"].copy()

In [ ]:
train

In [ ]:
train.info()

In [ ]:
train

# EDA

## Смотрим на train и test

In [ ]:
train = train.sort_values(["date", "time"])
test = test.sort_values(["date", "time"])

In [ ]:
train = train.reset_index().rename(columns={"index": "id"})
train

In [ ]:
train["target"].describe()

In [ ]:
train = prepare_dataset(train)
test = prepare_dataset(test)

In [ ]:
train

In [ ]:
# train[train.date == "2020-09-04"][["datetime", "target"]].head(20).plot(x="datetime", y="target")

In [ ]:
# train[train.date == "2020-09-05"][["datetime", "target"]].head(20).plot(x="datetime", y="target")

In [ ]:
fig = px.line(train.sort_values("datetime"), x="datetime", y="target", hover_data=["date", "time"])
fig.show()

In [ ]:
train["date"].min(), train["date"].max()

In [ ]:
test["date"].min(), test["date"].max()

In [ ]:
train["time"].unique()

In [ ]:
train["weather_pred"][train["weather_pred"].fillna("").str.contains("ясн")].unique()

In [ ]:
train["weather_pred"].str.contains("ясн").sum()

In [ ]:
train["weather_fact"].value_counts()

In [ ]:
train['datetime'].dt.year.unique()

In [ ]:
plot_target_boxplots(train)

In [ ]:
# Из Kaggle, покомпонентое разложение, надо заставить его нормально работать

period = 365

# Multiplicative Decomposition
multiplicative_decomposition = seasonal_decompose(
    train["target"], model="multiplicative", period=period
)

# Additive Decomposition
additive_decomposition = seasonal_decompose(train["target"], model="additive", period=period)

# Plot
# plt.rcParams.update({"figure.figsize": (15, 15), "figure.dpi": 70})
multiplicative_decomposition.plot().suptitle("Multiplicative Decomposition", fontsize=16)
plt.tight_layout(rect=(0, 0.03, 1, 0.95))

additive_decomposition.plot().suptitle("Additive Decomposition", fontsize=16)
plt.tight_layout(rect=(0, 0.03, 1, 0.95))

plt.show()

Ошибка прогноза температуры

In [ ]:
print((train['temp_pred'] - train['temp']).describe())
(train['temp_pred'] - train['temp']).hist(bins=50)
plt.title('Ошибка прогноза температуры')
plt.show()

## Стационарность

In [ ]:
daily_target = train.groupby(pd.Grouper(key='datetime', freq='D'))['target'].sum()

In [ ]:
monthly_target = train.groupby(pd.Grouper(key='datetime', freq='MS'))['target'].sum()

In [ ]:
from src.plots import plot_time_series

In [ ]:
plot_time_series(daily_target, 30, "Daily")

In [ ]:
plot_time_series(monthly_target, 12, "Monthly")

Тест на стационарность

In [ ]:
from src.plots import test_stationarity

In [ ]:
test_stationarity(daily_target)

## Корреляции

In [ ]:
num_features = ['temp', 'temp_pred']
sample = pd.concat([train, test])

In [ ]:
sample[num_features + ['target']].corr()

## Погода в данных vs спарсенная 

In [ ]:
train.shape

In [ ]:
weather = prepare_parsed_weather(datasets['weather_parsed'])
weather

In [ ]:
comparison = train.merge(weather, on = 'datetime')

In [ ]:
weather[weather.datetime == '2019-01-01 11:00:00']

In [ ]:
(comparison['weather_fact'].str.contains('дожд') == comparison['weather_category_parsed'].str.contains('дожд')).value_counts()

In [ ]:
train['weather_fact'].value_counts()

In [ ]:
comparison

In [ ]:
train['datetime']

In [ ]:
datasets['weather_parsed']['Cl'].value_counts()

# Baseline

## `target` ровно год назад

In [ ]:
train[train.date == "2020-01-01"].head()  # с id 8760 должны появиться предсказания baseline модели

In [ ]:
train.shape[0]

In [ ]:
test.shape[0]

In [ ]:
test.shape[0] / (train.shape[0] + test.shape[0])

In [ ]:
train.date.unique().shape[0]

In [ ]:
test.date.unique().shape[0]

In [ ]:
model = BaselineYearAgo()
model.fit(train.drop("target", axis=1), train["target"])

# Пример невычислимого предсказания
# df = train[train.date >= '2020-12-31'].drop('target', axis=1)
# df['predict'] = model.predict(df)

In [ ]:
# df = train[train.date >= '2020-01-01'] # с этой даты baseline модель может выдать прогноз
# df['predict'] = model.predict(df.drop('target', axis=1))
# train_metrics = pd.json_normalize(evaluate(df['target'], df['predict']))
# train_metrics

In [ ]:
tracker.add_experiment(
    model,
    train[train.date >= "2020-01-01"],
    test,
    "Baseline: значение год назад относительно текущей даты",
)

In [ ]:
train[train.date >= "2020-01-01"].date.unique().shape

In [ ]:
df2 = aggregated_daily_predictions(tracker.experiments[0]["train"]).reset_index()
pd.json_normalize(evaluate(df2["target"], df2["predict"]))

In [ ]:
plot_prediction(tracker.experiments[0]["train"], "Train: hourly")
plot_prediction(df2, "Train: daily")

In [ ]:
pd.json_normalize(evaluate(df2["target"], df2["predict"]))

In [ ]:
plot_prediction(tracker.experiments[0]["test"], "Test: hourly")
plot_prediction(df2, "Test: daily")

# Prophet

## Baseline

In [ ]:
train = prepare_dataset(datasets["source_train"].copy())
test = prepare_dataset(datasets["source_test"].copy())

In [ ]:
prophetBaseline = Prophet()
prophetBaseline.fit(train)

In [ ]:
tracker.add_experiment(prophetBaseline, train, test, "Prophet baseline")

In [ ]:
forecast = test.merge(prophetBaseline.predict(test), on="ds")
forecast["predict"] = forecast["yhat"]
forecast

In [ ]:
pd.json_normalize(evaluate(forecast["target"], forecast["predict"]))

In [ ]:
df2 = aggregated_daily_predictions(forecast)
pd.json_normalize(evaluate(df2["target"], df2["predict"]))

## Add regressors: Температура

In [ ]:
train.temp_pred.isna().sum()

In [ ]:
train["temp_pred"] = train["temp_pred"].ffill()

In [ ]:
test["temp_pred"] = test["temp_pred"].ffill()

In [ ]:
prophet2 = Prophet()
prophet2.add_regressor("temp")
prophet2.add_regressor("temp_pred")
prophet2.fit(train[["ds", "temp", "temp_pred", "y"]])

In [ ]:
tracker.add_experiment(prophet2, train, test, "Prophet with temperature")

In [ ]:
# prophet2.plot_components(forecast)

## Add regressors: Температура, праздники и население

In [ ]:
train = train.merge(datasets["population"], left_on=train["datetime"].dt.year, right_on="year")
test = test.merge(datasets["population"], left_on=test["datetime"].dt.year, right_on="year")

In [ ]:
datasets["population"]

In [ ]:
prophet3 = Prophet()
prophet3.add_regressor("temp")
prophet3.add_regressor("temp_pred")
prophet3.add_regressor("population")
# prophet3.add_country_holidays('RU')
prophet3.fit(train[["ds", "temp", "temp_pred", "population", "y"]])

In [ ]:
tracker.add_experiment(prophet3, train, test, "Prophet with temperature, holidays and population 2")

In [ ]:
tracker.display_metrics()

In [ ]:
tracker.get_best_experiment()['model']

In [ ]:
tracker.save_to_pickle()

In [ ]:
ExperimentTracker.load_experiment().display_metrics()